In [19]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold,StratifiedKFold
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
import random
import re
import torch

In [57]:
df = pd.read_csv('./n2_pred.csv')

In [58]:
p_mask = df['prob'] > 0.7
n_mask = df['prob'] < 0.5

In [59]:
df_mix = pd.concat([df[p_mask][:50],df[n_mask][:50]])

In [60]:
labels = [1 for i in range(50)]
labels.extend(0 for i in range(50))

In [61]:
df_mix.insert(loc=0, column='label', value=labels)

In [62]:
df_mix

,label,cid,iupac,desc,smiles,pred,prob
35,1,65983216,3-[[N-methyl-2-(propylaminomethyl)anilino]meth...,"The molecule contains methyl, propylaminomethy...",CCCNCc1ccccc1N(C)CC1CC(O)C1,1.0,0.765831
57,1,17173343,"2,3-dichloro-N-(2-methoxyethyl)benzamide","The molecule contains dichloro, methoxyethyl, ...",COCCNC(=O)c1cccc(Cl)c1Cl,1.0,0.787722
87,1,140960198,"2,2,3,3,5,5,6,6-octadeuterio-1-(4-nitrophenyl)...","The molecule contains octadeuterio, nitropheny...",O=[N+]([O-])c1ccc(N2CCNCC2)cc1,1.0,0.807710
119,1,43267355,N-cyclopentyl-N-methyl-2-piperidin-4-ylacetamide,"The molecule contains cyclopentyl, methyl, pip...",CN(C(=O)CC1CCNCC1)C1CCCC1,1.0,0.745748
136,1,115648108,"2-[1-[(5-methyl-1,2,4-oxadiazol-3-yl)methyl]py...","The molecule contains methyl, oxadiazol, yl, m...",Cc1nc(CN2CCC(CCO)C2)no1,1.0,0.938234
...,...,...,...,...,...,...,...
48,0,1685264,"4-[[4-chloro-1-(2-chlorophenyl)-2,5-dioxopyrro...","The molecule contains chloro, chlorophenyl, di...",CCN(C(=O)c1ccc(NC2=C(Cl)C(=O)N(c3ccccc3Cl)C2=O...,0.0,0.022639
49,0,87521185,10-methyl-9-(4-pyrrolidin-1-ylphenyl)acridin-1...,"The molecule contains methyl, pyrrolidin, ylph...",C[n+]1c2ccccc2c(-c2ccc(N3CCCC3)cc2)c2ccccc21,0.0,0.017791
50,0,88685024,1-methoxyethyl (E)-2-methylbut-2-enoate,"The molecule contains methoxyethyl, methylbut,...",CC=C(C)C(=O)OC(C)OC,0.0,0.226466
51,0,103804028,"1-[4-bromo-2-(1,2-oxazol-5-ylmethoxy)phenyl]et...","The molecule contains bromo, oxazol, ylmethoxy...",CC(=O)c1ccc(Br)cc1OCc1ccno1,0.0,0.222824


In [63]:
df_mix = df_mix.sample(frac=1)

In [64]:
df_mix

,label,cid,iupac,desc,smiles,pred,prob
25,0,69218153,disodium;(4-chlorophenyl)-phosphonatomethanone,"The molecule contains disodium, chlorophenyl, ...",O=C(c1ccc(Cl)cc1)P(=O)([O-])[O-],0.0,0.155979
431,1,79843913,N-[[1-[[methyl(2-methylsulfonylethyl)amino]met...,"The molecule contains methyl, methylsulfonylet...",CN(CCS(C)(=O)=O)CC1(CNC2CC2)CCCCCC1,1.0,0.813128
3,0,112185103,N'-(2-cyclopropylethyl)-N-ethyl-3-(pyrrolidin-...,"The molecule contains N', cyclopropylethyl, et...",CCNC(=NCCC1CC1)N1CCC(CN2CCCC2)C1,0.0,0.079158
26,0,103322558,"3-[(2-chlorothieno[2,3-d]pyrimidin-4-yl)amino]...","The molecule contains chlorothieno, pyrimidin,...",CCC(CCO)Nc1nc(Cl)nc2sccc12,0.0,0.182121
700,1,80352293,"N-[(2-fluorophenyl)methyl]-2-(1-methyl-1,2,4-t...","The molecule contains fluorophenyl, methyl, me...",Cn1cnc(CCNCc2ccccc2F)n1,1.0,0.799074
...,...,...,...,...,...,...,...
766,1,64095063,4-amino-1-[4-[2-(dimethylamino)acetyl]piperazi...,"The molecule contains amino, dimethylamino, ac...",CC(N)CCC(=O)N1CCN(C(=O)CN(C)C)CC1,1.0,0.788656
29,0,138891790,"ethyl 2-[prop-2-enyl-(2-sulfanylidene-3H-1,3,4...","The molecule contains ethyl, prop, enyl, sulfa...",C=CCN(CC(=O)OCC)c1n[nH]c(=S)s1,0.0,0.257996
48,0,1685264,"4-[[4-chloro-1-(2-chlorophenyl)-2,5-dioxopyrro...","The molecule contains chloro, chlorophenyl, di...",CCN(C(=O)c1ccc(NC2=C(Cl)C(=O)N(c3ccccc3Cl)C2=O...,0.0,0.022639
52,0,36099977,2-[(4-ethoxyphenyl)methyl-methylamino]-N-methy...,"The molecule contains ethoxyphenyl, methyl, me...",CCOc1ccc(CN(C)CC(=O)N(C)c2ccc([N+](=O)[O-])cc2...,0.0,0.489543


In [65]:
smiles_list = df_mix['smiles'].values
y = df_mix['label'].values
sum(y==0)

50

In [66]:
x = []
for smiles in smiles_list:
    mol=Chem.MolFromSmiles(smiles)
    fp = MACCSkeys.GenMACCSKeys(mol)
    # fp_bits = fp.ToBitString()
    fp_bits = fp.ToList()
    x.append(fp_bits)
x = np.array(x)
x.shape

(100, 167)

In [67]:
sfolder = StratifiedKFold(n_splits=3,random_state=0,shuffle=True)

In [68]:
f1_list = []
precision_list =  []
for train, test in sfolder.split(x,y):
    # print(sum(y[test]==0))
    # print(sum(y[test]==1))
    regr = make_pipeline(SVC(probability = True))
    regr.fit(x[train], y[train])
    pred = regr.predict(x[test])
    # print(pred)
    # print(y[test])
    f1 = f1_score(y[test].reshape(-1,1), pred.reshape(-1,1), average='micro')
    p_score = precision_score(y[test], pred, average='micro')
    f1_list.append(f1)
    precision_list.append(p_score)
    # print(f"f1: {f1}; precision: {p_score}.")

f1_ts = torch.Tensor(f1_list)
precision_ts = torch.Tensor(precision_list)

print(f"F1 mean: {f1_ts.mean()}, std: {f1_ts.std()}.")
print(f"Precision mean: {precision_ts.mean()}, std: {precision_ts.std()}.")


F1 mean: 0.7804515361785889, std: 0.041940029710531235.
Precision mean: 0.7804515361785889, std: 0.041940029710531235.


In [69]:
import joblib
joblib.dump(regr, "SVC.m")

['SVC.m']

# 加入额外的数据

In [70]:
df_positive = pd.read_csv('./data/positive.csv')
smiles_positive = df_positive['smiles'].values
y_positive = df_positive['label'].values

df_negtive = pd.read_csv('./data/negtive.csv')
smiles_negtive = df_negtive['smiles'].values
y_negtive = df_negtive['label'].values

In [71]:
x_positive = []
y_positive = []
################# 正样本
for smiles in smiles_positive:
    mol=Chem.MolFromSmiles(smiles)
    fp = MACCSkeys.GenMACCSKeys(mol)
    # fp_bits = fp.ToBitString()
    fp_bits = fp.ToList()
    x_positive.append(fp_bits)
    y_positive.append(1)
x_positive = np.array(x_positive)
y_positive = np.array(y_positive)

x_negtive = []
y_negtive = []
################# 负样本
for smiles in smiles_negtive:
    mol=Chem.MolFromSmiles(smiles)
    fp = MACCSkeys.GenMACCSKeys(mol)
    # fp_bits = fp.ToBitString()
    fp_bits = fp.ToList()
    x_negtive.append(fp_bits)
    y_negtive.append(0)
x_negtive = np.array(x_negtive)
y_negtive = np.array(y_negtive)

In [72]:
idx_positive = [i for i in range(len(x_positive))]
idx_negtive = [i for i in range(len(x_negtive))]

In [73]:
sample_list = [5,10,15,20,25,30,35,40,45,50]
# sample_list = [100,200,300,400]

f1_score_mean = []
f1_score_std = []
for sample_num in sample_list:
    random.seed(101)

    idx_p = random.sample(idx_positive, sample_num)
    idx_n = random.sample(idx_negtive, sample_num)

    x_extra = np.concatenate([x_positive[idx_p],x_negtive[idx_n]])
    y_extra = np.concatenate([y_positive[idx_p],y_negtive[idx_n]])

    idx_s = [i for i in range(len(y_extra))]
    random.shuffle(idx_s)
    x_extra = x_extra[idx_s]
    y_extra = y_extra[idx_s]

    f1_list = []
    precision_list =  []
    for train, test in sfolder.split(x,y):
        regr = make_pipeline(SVC(probability = True))
        regr.fit(np.concatenate([x[train], x_extra]), np.concatenate([y[train], y_extra]))
        pred = regr.predict(x[test])
        f1 = f1_score(y[test].reshape(-1,1), pred.reshape(-1,1), average='micro')
        p_score = precision_score(y[test], pred, average='micro')

        f1_list.append(f1)
        precision_list.append(p_score)
        # print(f"f1: {f1}; precision: {p_score}.")

    f1_ts = torch.Tensor(f1_list)
    precision_ts = torch.Tensor(precision_list)

    f1_score_mean.append(round(f1_ts.mean().item(),4))
    f1_score_std.append(round(f1_ts.std().item(),4))

f1_score_mean

[0.8099,
 0.8304,
 0.8604,
 0.8604,
 0.8503,
 0.8797,
 0.8601,
 0.8402,
 0.8702,
 0.8601]